In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import requests
import json
from pprint import pprint

TMDB_API_KEY = 'd43c34da8b6513938295a31cd66258b2'

def get_movie_datas():
    total_data = []

    # 1페이지부터 500페이지까지 (페이지당 20개, 총 10,000개)
    for i in range(1, 100):
        request_url = "https://api.themoviedb.org/3/movie/popular"
        params = {
            "api_key" : TMDB_API_KEY,
            "language" : "ko-KR",
            "page" : i,
        }
        # request_url = f"<https://api.themoviedb.org/3/movie/popular?api_key={TMDB_API_KEY}&language=ko-KR&page={i}>"
        movies = requests.get(request_url, params=params).json()
        pprint(movies)
        for movie in movies['results']:
            # print(movie['genre_ids'])
            # print(type(movie['genre_ids']))

            if movie.get('release_date', '') and movie.get('overview', ''):
                fields = {
                    # 'movie_id': movie['id'],
                    'title': movie['title'],
                    'released_date': movie['release_date'],
                    # 'popularity': movie['popularity'],
                    'vote_avg': movie['vote_average'],
                    'overview': movie['overview'],
                    'poster_path': movie['poster_path'],
                    'genres': movie['genre_ids']
                }

                data = {
                    "pk": movie['id'],
                    "model": "movies.movie",
                    "fields": fields
                }

                total_data.append(data)

    with open("movie_data_base.json", "w", encoding="utf-8") as w:
        json.dump(total_data, w, indent=4, ensure_ascii=False)

get_movie_datas()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
              'title': 'Megan',
              'video': False,
              'vote_average': 0,
              'vote_count': 0},
             {'adult': False,
              'backdrop_path': '/lOXVbF2mBK0xUt8sLInF3TIk5YS.jpg',
              'genre_ids': [28, 878],
              'id': 2048,
              'original_language': 'en',
              'original_title': 'I, Robot',
              'overview': '2035년, 인간은 지능을 갖춘 로봇에게 생활의 모든 편의를 제공받으며 편리하게 살아가게 '
                          '된다. 인간의 안전을 최우선으로 하는 로봇 3원칙이 내장된 로봇은 인간을 위해 요리하고 '
                          '아이들을 돌보며 없어서는 안될 신뢰 받는 동반자로 여겨진다. NS-4에 이어 더 높은 지능을 '
                          '가진 로봇 NS-5의 출시를 하루 앞둔 어느 날, NS-5의 창시자인 래닝 박사가 의문의 '
                          '죽음을 맞이한다. 그의 죽음을 둘러싼 수많은 추측이 난무한 가운데, 시카고 경찰 델 '
                          '스프너(윌 스미스)는 자살이 아니라는데 확신을 갖고 조사에 착수한다. 끔찍한 사고 이후 '
                          '로봇에 대한 적대감을 갖고 있던 그는 이 사건 역시 로봇과 관련이 있다고 믿고 숨은 음모를 '
                          '파헤치는데...',
  

In [ ]:
import json
import csv

In [ ]:
# with open('/content/movie_data.json', 'r', encoding = 'utf-8') as input_file, open('melon.csv', 'w', newline = '') as output_file :
#     data = json.load(input_file)

In [ ]:
movie_json = pd.read_json('/content/movie_data_base.json')

In [ ]:
movie_json

,pk,model,fields
0,436270,movies.movie,"{'title': '블랙 아담', 'released_date': '2022-10-1..."
1,882598,movies.movie,"{'title': '스마일', 'released_date': '2022-09-23'..."
2,505642,movies.movie,"{'title': '블랙 팬서: 와칸다 포에버', 'released_date': '..."
3,829280,movies.movie,"{'title': '에놀라 홈즈 2', 'released_date': '2022-1..."
4,979924,movies.movie,"{'title': '온 더 라인', 'released_date': '2022-10-..."
...,...,...,...
1620,522162,movies.movie,"{'title': '미드웨이', 'released_date': '2019-11-06..."
1621,812456,movies.movie,"{'title': '바비와 첼시: 사라진 생일', 'released_date': '..."
1622,274167,movies.movie,"{'title': '대디스 홈', 'released_date': '2015-12-2..."
1623,10528,movies.movie,"{'title': '셜록 홈즈', 'released_date': '2009-12-2..."


In [ ]:
movie_pk = movie_json['pk']

In [ ]:
movie_pk

0       436270
1       882598
2       505642
3       829280
4       979924
         ...  
1620    522162
1621    812456
1622    274167
1623     10528
1624     77951
Name: pk, Length: 1625, dtype: int64

In [ ]:
data = pd.json_normalize(movie_json['fields'])

In [ ]:
data = pd.concat([movie_pk,data], axis=1)

In [ ]:
type(data)

pandas.core.frame.DataFrame

In [ ]:
data[0:1]

,pk,title,released_date,vote_avg,overview,poster_path,genres
0,436270,블랙 아담,2022-10-19,6.9,기원전 가장 번성하고 위대한 고대 국가였지만 현재는 국제 군사 조직 인터갱의 독재 ...,/mEdMHGy1FfCUc7PskFO0tibm8jp.jpg,"[28, 14, 878]"


In [ ]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape)


TF-IDF 행렬의 크기(shape) : (1625, 37579)


In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)
  

코사인 유사도 연산 결과 : (1625, 1625)


In [ ]:
title_to_index = dict(zip(data['pk'], data.index))

# 영화 pk 436270(블랙아담)의 인덱스를 리턴
idx = title_to_index[436270]
print(idx)

0


In [ ]:
def get_recommendations(pk, cosine_sim=cosine_sim):
    # 선택한 영화의 pk로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[pk]

    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 영화의 pk 값을 리턴한다.
    return data['pk'].iloc[movie_indices]

In [ ]:
type(get_recommendations(436270))

pandas.core.series.Series

In [ ]:
result = get_recommendations(316727)
result.tolist()

[158015, 602223, 442249, 999205, 10439, 22970, 567609, 131631, 84892, 70981]

In [ ]:
recommend_list = []
for i in range(len(data)):
  
  recommend_list.append(get_recommendations(movie_pk[i]).tolist())

In [ ]:
recommend_list

[[354556, 43641, 351819, 299536, 675353, 464052, 176, 100402, 39100, 453395],
 [9358, 8643, 19912, 999205, 760741, 333484, 343611, 256274, 482373, 774741],
 [595743, 18861, 763788, 926899, 714888, 619778, 57158, 2048, 220289, 176983],
 [807070,
  497582,
  610150,
  916740,
  953300,
  516329,
  763788,
  1040263,
  20526,
  38575],
 [567609, 799876, 899112, 274167, 458576, 72331, 848058, 880313, 414906, 215],
 [595586, 706503, 10555, 400928, 637649, 101, 1013228, 760868, 663, 260346],
 [672322, 509967, 13186, 220289, 743439, 778552, 785538, 381284, 629542, 630],
 [399178,
  1018647,
  424,
  675327,
  937013,
  768449,
  773975,
  287590,
  774741,
  497582],
 [899112,
  504253,
  755566,
  764835,
  59440,
  523936,
  589761,
  14836,
  999722,
  585245],
 [785985, 82507, 220289, 773975, 698128, 567609, 260346, 595743, 9637, 11324],
 [14248, 489430, 840882, 597891, 13179, 346364, 531219, 372058, 13394, 405774],
 [14160, 4523, 668461, 791177, 555604, 779782, 616037, 830788, 877269, 14

In [ ]:
recommend_list[0]

[354556, 43641, 351819, 299536, 675353, 464052, 176, 100402, 39100, 453395]

In [ ]:
import requests
import json
from pprint import pprint

# recommend 열에 추천 영화 상위 10개 목록
TMDB_API_KEY = 'd43c34da8b6513938295a31cd66258b2'

def get_movie_datas():
    total_data = []
    # 1페이지부터 500페이지까지 (페이지당 20개, 총 10,000개)
    for i in range(1, 100):
        request_url = "https://api.themoviedb.org/3/movie/popular"
        params = {
            "api_key" : TMDB_API_KEY,
            "language" : "ko-KR",
            "page" : i,
        }
        # request_url = f"<https://api.themoviedb.org/3/movie/popular?api_key={TMDB_API_KEY}&language=ko-KR&page={i}>"
        movies = requests.get(request_url, params=params).json()
        pprint(movies)
        index = 0
        for movie in movies['results']:
            
            # print(movie['genre_ids'])
            # print(type(movie['genre_ids']))
            

  
            
            if movie.get('release_date', '') and movie.get('overview', ''):
                fields = {
                    # 'movie_id': movie['id'],
                    'title': movie['title'],
                    'released_date': movie['release_date'],
                    # 'popularity': movie['popularity'],
                    'vote_avg': movie['vote_average'],
                    'overview': movie['overview'],
                    'poster_path': movie['poster_path'],
                    'genres': movie['genre_ids'],
                    'recommend' : get_recommendations(movie['id']).tolist()
                }

                index += 1
                data = {
                    "pk": movie['id'],
                    "model": "movies.movie",
                    "fields": fields
                }

                total_data.append(data)
    with open("movie_data.json", "w", encoding="utf-8") as w:
        json.dump(total_data, w, indent=4, ensure_ascii=False)

get_movie_datas()


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
              'title': 'Megan',
              'video': False,
              'vote_average': 0,
              'vote_count': 0},
             {'adult': False,
              'backdrop_path': '/lOXVbF2mBK0xUt8sLInF3TIk5YS.jpg',
              'genre_ids': [28, 878],
              'id': 2048,
              'original_language': 'en',
              'original_title': 'I, Robot',
              'overview': '2035년, 인간은 지능을 갖춘 로봇에게 생활의 모든 편의를 제공받으며 편리하게 살아가게 '
                          '된다. 인간의 안전을 최우선으로 하는 로봇 3원칙이 내장된 로봇은 인간을 위해 요리하고 '
                          '아이들을 돌보며 없어서는 안될 신뢰 받는 동반자로 여겨진다. NS-4에 이어 더 높은 지능을 '
                          '가진 로봇 NS-5의 출시를 하루 앞둔 어느 날, NS-5의 창시자인 래닝 박사가 의문의 '
                          '죽음을 맞이한다. 그의 죽음을 둘러싼 수많은 추측이 난무한 가운데, 시카고 경찰 델 '
                          '스프너(윌 스미스)는 자살이 아니라는데 확신을 갖고 조사에 착수한다. 끔찍한 사고 이후 '
                          '로봇에 대한 적대감을 갖고 있던 그는 이 사건 역시 로봇과 관련이 있다고 믿고 숨은 음모를 '
                          '파헤치는데...',
  